In [1]:
import matplotlib as mpl
mpl.use("pgf")
# from palettable.tableau import Tableau_10
# from cycler import cycler
import numpy as np
pgf_with_pdflatex = {
    "pgf.texsystem": "pdflatex",
    "pgf.preamble": [
         r"\usepackage[utf8x]{inputenc}",
         r"\usepackage[T1]{fontenc}",
         r"\usepackage{mathptmx}",
         r"\usepackage{mathtools}",
#          r"\usepackage{biblatex}",
         ],
    "text.usetex": True,
    "figure.figsize": [4, 3.5],
    "axes.labelsize": 12,
    "font.size": 12,
    "font.family": "serif",
    "legend.fontsize": 10,
    "xtick.labelsize": 10,
    "ytick.labelsize": 10,
    "lines.linewidth": 1.5,
#     "axes.prop_cycle": cycler(color=Tableau_10.mpl_colors),
    "xtick.major.size": 5,
    "xtick.major.width": 1,
    "xtick.minor.size": 2.5,
    "xtick.minor.width": 1,
    "ytick.major.size": 5,
    "ytick.major.width": 1,
    "ytick.minor.size": 2.5,
    "ytick.minor.width": 1,
}
mpl.rcParams.update(pgf_with_pdflatex)

import gspread
from oauth2client.service_account import ServiceAccountCredentials
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.legend_handler import HandlerLine2D
from matplotlib.ticker import FormatStrFormatter
# from palettable.tableau import Tableau_10
from cycler import cycler
from pathlib import Path

In [2]:
phi_1_files = [Path('pressure-traces')/p for p in [Path('Tc_700K_P0_2.1848_T0_353K_pressure.txt'), Path('Tc_734K_P0_1.8013_T0_353K_pressure.txt'), Path('Tc_775K_P0_1.4365_T0_353K_pressure.txt')]]
phi_2_files = [Path('pressure-traces')/p for p in [Path('Tc_699K_P0_0.7082_T0_373K_pressure.txt'), Path('Tc_723K_P0_0.7694_T0_373K_pressure.txt'), Path('Tc_786K_P0_0.7856_T0_413K_pressure.txt')]]

In [3]:
scope = ['https://spreadsheets.google.com/feeds']
credentials = ServiceAccountCredentials.from_json_keyfile_name('methyl-valerate-78135e2c2c7c.json',
                                                               scope)
gc = gspread.authorize(credentials)
ws = gc.open_by_key('1UFdGzEyx7PsKprPybi-lkbuWYS8BAS-cBm2IKSKY-UA').sheet1
recs = ws.get_all_records(empty2zero=True)
data = pd.DataFrame(recs)

phi_200_15_bar = data[(data['Equivalence Ratio'] == 2.00) &
                      (np.isclose(data['Compressed Pressure (bar)'], 15, rtol=1E-2))]
phi_100_30_bar = data[(data['Equivalence Ratio'] == 1.00) &
                      (np.isclose(data['Compressed Pressure (bar)'], 30, rtol=1E-2))]

phi_1_temps = phi_100_30_bar['Compressed Temperature (K)'].iloc[[1, 3, 5]]
phi_1_comp_times = phi_100_30_bar['Compression Time (ms)'].iloc[[1, 3, 5]]
phi_1_comp_times.iloc[-1] -= 1.0
phi_2_temps = phi_200_15_bar['Compressed Temperature (K)'].iloc[[3, 5, 11]]
phi_2_comp_times = phi_200_15_bar['Compression Time (ms)'].iloc[[3, 5, 11]]

In [4]:
fig, ax_phi_1 = plt.subplots()

for i, f in enumerate(phi_1_files):
    data = np.loadtxt(str(f))
    ax_phi_1.plot(data[:, 0]*1000 - phi_1_comp_times.iloc[i], data[:, 1], label='$T_C = {}$ K'.format(phi_1_temps.iloc[i]))
    
nrdata = np.loadtxt('pressure-traces/NR_Tc_700K_P0_2.1848_T0_353K_pressure.txt')
ax_phi_1.plot(nrdata[:, 0]*1000 + 0.25, nrdata[:, 1], label='Non-Reactive')

ax_phi_1.minorticks_on()

ax_phi_1.set_xlim(-20, 15)
ax_phi_1.set_ylim(0, 60)
ax_phi_1.set_ylabel('Pressure, bar')
ax_phi_1.set_xlabel('Time, ms')
ax_phi_1.legend(loc='upper center')
# ax_phi_1.text(0.04, 0.9, 'a)', transform=ax_phi_1.transAxes, fontweight='bold')

In [5]:
fig.savefig('pressure-traces.pgf', bbox_inches='tight')